<a href="https://colab.research.google.com/github/rvraghvender/DeepLearningProjects/blob/main/NaturalLanguageProcessing/BuildingRNN/Building_a_Recurrent_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Builiding Recurrent Neural Network

Recurrent Neural Network (RNN) are very effective for Natural Language Processing and other sequence tasks because they have "memory". They can read input $x^{<t>}$ (such as words) one at a time, and remeber some contextual information through the hidden layer activation that get passed from one step to the next. This allows a unidirectional (one-way) RNN to take information from the past to process later inputs. A bidrectional (two-way) RNN can text context from both the past and the future, much like Marty McFly (Movie: Back to the Future).

## Notation:

- Superscript $[l]$ denotes an object associated with the $l^{th}$ layer.
- Superscript $(i)$ denotes an object associated with the $i^{th}$ example.
- Superscript $<t>$ denotes an object at the $t^{th}$ time step.
- Subscript $i$ denotes the $i^{th}$ entry of a vector.

**Example**
- $a_5^{(2)[3]<4>}$ denotes the activation of the 2nd training example (2), 3rd layer [3], 4th time step <4>, and 5th entry in the vector.

In [1]:
# importing relevant packages
import numpy as np

## Forward Propagation for the Basic Recurrent Neural Network

The basic RNN has the following structure:
  
In this example, $T_x$ = $T_y$
<img src="https://github.com/rvraghvender/DeepLearningProjects/blob/main/NaturalLanguageProcessing/BuildingRNN/images/rnn.png?raw=true" style="width:500;height:300px;">
<caption><center><font color='purple'><b>Figure 1</b>: Basic RNN model </center></caption>

## Dimension of input $x$
### Input with $n_x$ number of units
- For a single time step of a single input example $x^{(i)<t>}$ is a one-dimensional input vector
- Using a language as an example, a language with 5000-word vocabulary could be one-hot encoded into a vector that has 5000 units. So $x^{(i)<t>}$ would have the shape (5000,)
- The notation, $n_x$, is used here to denote the number of units in a single time step of a single training example.

### Time steps of size $T_x$
- A recurrent neural network has multiple time steps, with the index $t$.
- $T_x$ denotes the number of timsteps in the longest sequence.

### Batches of size $m$
- Consider mini-batches, each with 20 training examples
- To benefit from vectorization, we will stack 20 columns of $x^{(i)}$ examples
- This tensor, has the shape (5000, 20, 10)
- Here $m$ is used to denote the number of training examples
- Therefore, the shape of mini-batch is ($n_x$, $m$, $T_x$)

### 3D Tensor of shape ($n_x$, $m$, $T_x$)
- The 3-dimensional tensor of $x$ of shape ($n_x$, $m$, $T_x$) represents the input $x$ that is fed into the RNN

### Taking a 2D slice for each time step: $x^{<t>}$
- At each time step, we will use a mini-batch of training example (not just a single example)
- So, for each time step $t$, we will use a 2D slice of shape ($n_x$, $m$)
- This 2D slice is referred to as $x^{<t>}$. The variable name used in this code is `xt`.